In [ ]:
### ipfn
Trying Iterative Proportional Filter
from ipfn import ipfn
m = [[40, 30, 20, 10], [35, 50, 100, 75], [30, 80, 70, 120], [20, 30, 40, 50], [40, 30, 40, 50]]
m = np.array(m)
xip = np.array([150, 300, 400, 140, 10])
xpj = np.array([300, 200, 400, 90, 10])
# xpj = np.array([100, 400, 400, 100])

aggregates = [xip, xpj]
dimensions = [[0], [1]]

IPF = ipfn.ipfn(m, aggregates, dimensions, convergence_rate=1e-6)
m = IPF.iteration()
print(m)

df_ = (df_real[["flag_garage", "log_mq", "flag_air_conditioning"]].sample(2000, random_state = 11)
       .assign(flag_garage = lambda x: x["flag_garage"].astype(bool),
               flag_air_conditioning = lambda x: x["flag_air_conditioning"].astype(bool),
               log_mq_cut = lambda x: pd.cut(x["log_mq"], x["log_mq"].quantile(np.arange(0,1.1,.1))),
               total = 1)
        .dropna()
        .assign(log_mq_cat = lambda x: [u.right for u in x["log_mq_cut"]])
        .drop(columns = ["log_mq"]))

df_ = (df_real[["flag_garage", "log_mq", "flag_air_conditioning"]].sample(2000, random_state = 11)
       .assign(flag_garage = lambda x: x["flag_garage"].astype(bool),
               flag_air_conditioning = lambda x: x["flag_air_conditioning"].astype(bool),
               log_mq_cut = lambda x: pd.cut(x["log_mq"], x["log_mq"].quantile(np.arange(0,1.1,.1))),
               total = 1)
        .dropna()
        .assign(log_mq_cat = lambda x: [u.left for u in x["log_mq_cut"]])
        .drop(columns = ["log_mq", "log_mq_cut"]))

garage_freq = df_["flag_garage"].value_counts(normalize = True)
log_mq_cat_freq = df_["log_mq_cat"].value_counts(normalize = True)
ac_freq = df_["flag_air_conditioning"].value_counts(normalize = True)

aggregates = [garage_freq,log_mq_cat_freq,ac_freq]
dimensions = [["flag_garage"],["log_mq_cat"], ["flag_air_conditioning"]]

IPF = ipfn.ipfn(df_, aggregates = aggregates, dimensions = dimensions, convergence_rate=1e-5)
df_ipf = IPF.iteration()
df_
df_ipf
((df_.reset_index()[["flag_garage", "log_mq", "flag_air_conditioning"]] + 0.) - (df_ipf.reset_index()[["flag_garage", "log_mq", "flag_air_conditioning"]] + 0.))
df_ipf
df_ipf.sum()
m1 = [[0, 30, 20, 10], [0, 30, 100, 10], [0, 80, 100, 120], [20, 30, 20, 50]]
m1 = np.array(m1)
xip = np.array([10, 40, 20, 150])
xpj = np.array([20, 60, 400, 300])

aggregates = [xip, xpj]
dimensions = [[0], [1]]

IPF = ipfn.ipfn(m1, aggregates, dimensions, convergence_rate=1e-6)
m2 = IPF.iteration()
print(m2)
m
### Gensynthpop
from gensynthpop.conditional_attribute_adder import ConditionalAttributeAdder
df_real["CAP"]
agent_ids = list()
agent_neighborhoods = list()
agent_count = 0
for neighb_code, (neighb_total) in read_marginal_data(['population'], 'population').iterrows():
    agent_ids += [f"SA{i + agent_count:06d}" for i in range(neighb_total.iloc[0])]
    agent_neighborhoods += [neighb_code] * neighb_total.iloc[0]
    agent_count += neighb_total.iloc[0]
df_synthetic_population = pd.DataFrame(data=dict(agent_id=agent_ids, neighb_code=agent_neighborhoods))

In [1]:
import sys
sys.path += ["../src"]
import utils
from glob import glob
import matplotlib.pyplot as plt 
import seaborn as sns
from matplotlib.pyplot import subplots as sbp 
from importlib import reload
import jl_vae
import jl_nflows_geo_coordinates_2 as nfg
from jl_nflows_geo_coordinates import load_nf as load_dict

from _51_abm_functions import cod_prov_abbrv_df

# Global Spatial Autocorrelation
from spatial_autocorrelation import get_moransI, moransI_scatterplot, hypothesis_testing
# Local Spatial Autocorrelation
from spatial_autocorrelation import get_localMoransI, LISA_scatterplot
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
from sklearn.preprocessing import OneHotEncoder
from jl_synthetic_pop_all_provinces import add_cat_features
from ipfn import ipfn
import config


In [170]:
cod_prov_abbrv_df.sort_values("prov_abbrv")

,prov_abbrv,COD_PROV
7343,AG,084
845,AL,006
4792,AN,042
1180,AO,007
4894,AP,044
...,...,...
3403,VE,027
3136,VI,024
3038,VR,023
4967,VT,056


In [2]:

# # 40 seconds
# abm_path = "ISP_data_for_ABM/ISP_ABM_up_to_2024_08_0001.csv"
# data = pd.read_csv(jl_vae.path_intermediate + abm_path, index_col = 0)
geo_dict = jl_vae.load_geo_data()

# # these need to be added to the generated data for predicting with hedonic regression
# year_erogaz = [f'year_erogaz_{u}' for u in [u for u in range(2016, 2025)]] 
# # Trained Hedonic Regression
# train_path = '/data/housing/data/intermediate/HedonicRegressionABM/reg_risk_OMI_id.pkl'
# # Load ISP data for the ABM and clean it 
# data_path = '/data/housing/data/intermediate/ISP_data_for_ABM/ISP_ABM_up_to_2024_08.csv'
# isp = pd.read_csv(data_path,dtype=str,encoding='latin1',na_values=config.default_missing)

# drop_col = ['floor_numeric','NUM_MQ_SUPERF_COMM','COD_CAT_CATASTALE','year_erogaz']
# isp = isp.drop(columns=drop_col)
# isp = utils.CorrectTypes_ABM(isp)

In [164]:
data_path = '/data/housing/data/intermediate/ISP_data_for_ABM/ISP_ABM_up_to_2024_08.csv'
isp = pd.read_csv(data_path,dtype=str,encoding='latin1',na_values=config.default_missing)

# drop_col = ['floor_numeric','NUM_MQ_SUPERF_COMM','COD_CAT_CATASTALE','year_erogaz']
# isp = isp.drop(columns=drop_col)
# isp = utils.CorrectTypes_ABM(isp)

In [ ]:
# choose the province
prov = "TO"
# get the cod prov (number)
cod_prov = cod_prov_abbrv_df.query("prov_abbrv == @prov")["COD_PROV"].item()
real_pops = jl_vae.path_pop_synth + f"pop_samples/pop_real_with_hedonic_price"
df_real_ = pd.read_csv(real_pops + f"/pop_real_full_250110{prov}.csv", index_col = 0)
df_real_ = add_cat_features(df_real_)
df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))

In [97]:
np.random.seed(2)
ran_xy = np.random.uniform(low = df_real["x"].min(), high = df_real["x"].max(), size = 1000000), np.random.uniform(low = df_real["y"].min(), high = df_real["y"].max(), size = 1000000)

In [98]:
df_locations = utils.spatial_matching_ABM(pd.DataFrame({"x": ran_xy[0], "y": ran_xy[1]}), geo_dict["hydro_risk"], geo_dict["census"], 
                                          geo_dict["omi_og"], geo_dict["cap"]).query("prov_abbrv == @prov")

In [114]:
df_real = df_real_.assign(log_price = lambda x: x["price_corrected"], CAP = lambda x: x["CAP"].astype(str))
df_real = df_real[['flag_garage', 'flag_pertinenza', 'flag_air_conditioning',
       # 'flag_multi_floor', 
       'log_mq', 'log_price',
       # 'flag_air_conditioning_Missing', 
       # 'x', 'y', 
       'CAP',
       'energy_class', 
       'COD_CAT',
       'anno_costruzione']]

In [149]:
def transform_bins(df, vars):
    df1 = df.copy()
    for var in vars:
        df1[var + "_bin"] =  pd.qcut(df1[var], q = 5, labels = False)
        df1.drop(columns = [var], inplace = True)
    return df1

def transform_bins_to_real(sample_df, df_real, var):
    bins_values = pd.qcut(df_real[var], q = 5, labels = False, retbins = True)[1]
    sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]
    sample_df.drop(columns = [var + "_bin"], inplace = True)
    return sample_df

    

In [116]:
# df = transform_bins(df_real, ["x", "y", "log_mq", "log_price"])
df = transform_bins(df_real, ["log_mq", "log_price"])#.drop(columns = ["CAP"])

In [178]:
df_cap

,flag_garage,flag_pertinenza,flag_air_conditioning,CAP,energy_class,COD_CAT,anno_costruzione,log_mq_bin,log_price_bin
0,1.0,0.0,0.0,10023,NaN,A02,1985_2005,1,2.0
6,0.0,0.0,0.0,10023,NaN,A_01_07_08,1500_1965,4,3.0
16,1.0,0.0,0.0,10023,NaN,A_01_07_08,1985_2005,4,4.0
19,1.0,0.0,0.0,10023,NaN,A02,1985_2005,0,1.0
61,1.0,0.0,0.0,10023,NaN,A02,1965_1985,3,2.0
...,...,...,...,...,...,...,...,...,...
29784,0.0,1.0,1.0,10023,NaN,A03,1500_1965,0,0.0
29817,1.0,0.0,0.0,10023,NaN,A02,2005_2025,0,2.0
29909,1.0,0.0,0.0,10023,NaN,A02,1500_1965,0,1.0
29930,1.0,0.0,1.0,10023,NaN,A02,1965_1985,2,2.0


In [124]:
for cap in df_real["CAP"].unique():
    df_cap = df.query("CAP == @cap")
    N_cap = len(df_cap)
    df_counts = df_cap.value_counts().reset_index()
    cap_locs = df_locations.query("CAP == @cap").sample(n = N_cap)
    sample_df = df_counts.sample(n = N_cap, weights = df_counts["count"], replace = True).drop(columns = ["count"])
    break

In [125]:
sample_df[["x", "y"]] = cap_locs[["GEO_LONGITUDINE_BENE_ROUNDED", "GEO_LATITUDINE_BENE_ROUNDED"]]

In [134]:
bins_values = pd.qcut(df_real["log_mq"], q = 5, labels = False, retbins = True)[1]

In [ ]:
for var in ["log_mq", "log_price"]:
    sample_df[var] = [np.random.uniform(low = bins_values[u], high = bins_values[u + 1]) for u in sample_df[var + "_bin"]]

[4.6640889997784925,
 4.312184114965763,
 4.420877027684081,
 6.089095687072165,
 4.804228580087222,
 3.394590170476961,
 4.6773056207443675,
 3.6579979743866953,
 4.457826658213793,
 4.431980150803148,
 4.30614131232162,
 4.379380678154088,
 5.325545887903551,
 4.765459842244139,
 4.901121784755449,
 3.833020542237883,
 4.393149063460593,
 4.580296003674842,
 4.40574154038355,
 4.340142114487733,
 5.394463897731341,
 5.4373657597956,
 6.439575402118962,
 6.507299761413281,
 4.469436302110628,
 4.416834548372668,
 4.50766699984731,
 4.907053169667219,
 4.635431615036934,
 4.297391083076913,
 4.458403502001746,
 4.904220806056812,
 4.4271522264145595,
 4.350534722694575,
 4.23507748551069,
 4.276578433945611,
 4.392855194799426,
 4.344788886191344,
 5.1129546580277,
 4.740391737493933,
 4.5190325214968645,
 4.750982796585938,
 4.605529090594717,
 5.959703712554902,
 4.854595231426414,
 5.08150130414309,
 4.827630118632996,
 5.003968686660824,
 4.636905265807475,
 4.40068970153774,
 6.01

In [ ]:
for _,(cap,count) in CAPS_prov.reset_index().iterrows():
    break

In [102]:
df.columns

Index(['flag_garage', 'flag_pertinenza', 'flag_air_conditioning', 'CAP',
       'anno_costruzione', 'log_mq_bin', 'log_price_bin'],
      dtype='object')

In [103]:
def get_ipf_input(df):
    seed = df.groupby(list(df.columns)).size().reset_index(name="count")
    remove_vals = []
    aggregates = []
    dimensions = []
    for var in df.columns:
        agg = df.groupby(var).size()
        aggregates.append(agg)
        dimensions.append([var])
        remove_vals.append([var, list(agg[agg == 1].index)])
    return seed, aggregates, dimensions, remove_vals

In [104]:
def clean_df(df):
    remove_obs = []
    repeat = True
    rep = 0
    df_out = df.copy()
    while repeat:
        rep += 1
        new_removals = []
        for col in df.columns:
            val_counts = df_out[col].value_counts()
            remove_class = list(val_counts[val_counts == 1].index)
            new_removals += list(df_out[df_out[col].isin(remove_class)].index)
        remove_obs += new_removals
        df_out = df_out[~df_out.index.isin(remove_obs)]
        print(len(remove_obs), len(df_out))
        if new_removals == []:
            repeat = False

        if rep > 20:
            repeat = False
    return df_out, remove_obs

In [105]:
CAPS_prov = df_real["CAP"].value_counts()

In [106]:
for _,(cap,count) in CAPS_prov.reset_index().iterrows():
    break

In [112]:
if count > 1:
    # df = transform_bins(df_real.query("CAP == @cap").dropna().drop(columns = ["CAP"]), ["log_mq", "log_price"])
    df = transform_bins(df_real.dropna().drop(columns = ["CAP"]), ["log_mq", "log_price"])
    # cleaned_df, remove_obs = clean_df(df)
    cleaned_df = df.copy()
    seed, aggregates, dimensions, remove_vals = get_ipf_input(cleaned_df)
    model = ipfn.ipfn(original = seed, aggregates = aggregates,
             dimensions = dimensions, weight_col = "count",
             convergence_rate = 1e-5, max_iteration=500)
    result = model.iteration()

In [113]:
result.sort_values("count", ascending = False)

,log_price_bin,flag_garage,flag_pertinenza,flag_air_conditioning,anno_costruzione,log_mq_bin,count
0,0,0.0,0.0,0.0,1500_1965,0,1110
25,0,0.0,0.0,0.0,1965_1985,0,783
411,4,1.0,0.0,0.0,1965_1985,4,671
1,1,0.0,0.0,0.0,1500_1965,0,517
436,4,1.0,0.0,0.0,1985_2005,4,495
...,...,...,...,...,...,...,...
200,4,0.0,0.0,1.0,Missing,3,1
536,3,1.0,0.0,1.0,2005_2025,0,1
199,3,0.0,0.0,1.0,Missing,0,1
108,0,0.0,0.0,0.0,Missing,2,1


In [95]:
cleaned_df.value_counts().reset_index()

,flag_garage,flag_pertinenza,flag_air_conditioning,anno_costruzione,log_mq_bin,log_price_bin,count
0,1.0,0.0,0.0,2005_2025,4,4,56
1,0.0,0.0,0.0,1965_1985,0,0,51
2,0.0,0.0,0.0,1500_1965,0,0,44
3,1.0,0.0,0.0,1965_1985,4,4,33
4,1.0,0.0,0.0,2005_2025,0,0,33
...,...,...,...,...,...,...,...
228,0.0,1.0,0.0,1985_2005,2,2,1
229,0.0,1.0,0.0,1965_1985,4,4,1
230,0.0,1.0,0.0,1965_1985,3,0,1
231,0.0,1.0,0.0,1965_1985,1,0,1


In [65]:
seed, aggregates, dimensions, remove_vals = get_ipf_input(clean_df)

In [ ]:

# weights = result.df

In [ ]:
result["count"]

179

In [ ]:
df = transform_bins(df_real.dropna(), ["log_mq", "log_price"])
seed, aggregates, dimensions, remove_vals = get_ipf_input(df)

[['flag_garage', []],
 ['flag_pertinenza', []],
 ['flag_air_conditioning', []],
 ['CAP', []],
 ['energy_class', []],
 ['COD_CAT', []],
 ['anno_costruzione', []],
 ['log_mq_bin', []],
 ['log_price_bin', []]]

In [170]:
df = transform_bins(df_real, ["log_mq", "log_price"])#.drop(columns = ["CAP"])
# aggregates = [df[var].value_counts().sort_index() for var in df.columns]
# dimensions = list(df.columns)
seed = df.groupby(list(df.columns)).size().reset_index(name="count")
remove_vals = []
aggregates_ = []
dimensions = []
for var in df.columns:
    agg = df.groupby(var).size()
    aggregates_.append(agg)
    dimensions.append([var])
    remove_vals.append([var, list(agg[agg == 1].index)])
# aggregates = [df[col].value_counts().sort_index().values for col in df.columns]
# dimensions = [[col] for col in df.columns]

In [178]:
remove_obs = []
for var, idx in remove_vals:
    if len(idx) > 0:
        remove_obs += list(df[df[var].isin(idx)].index)

In [ ]:
39022

In [206]:
seed.query("CAP == '39022'")

,flag_garage,flag_pertinenza,flag_air_conditioning,CAP,energy_class,COD_CAT,anno_costruzione,log_mq_bin,log_price_bin,count
115,1.0,0.0,0.0,39022,Missing_energy_class,A02,1965_1985,0,0.0,1


In [179]:
remove_obs

[228, 363, 859]

In [162]:
agg = aggregates[3]

In [215]:
df = seed.copy()
steps = len(aggregates)
weight_col = "count"
if True:
        tables = [df]
        for inc in range(steps - 1):
            tables.append(df.copy())
        original = df.copy()

        # Calculate the new weights for each dimension
        inc = 0
        for features in dimensions:
            if inc == (steps - 1):
                table_update = df
                table_current = tables[inc].copy()
            else:
                table_update = tables[inc + 1]
                table_current = tables[inc]

            tmp = table_current.groupby(features)[weight_col].sum()
            xijk = aggregates[inc]

            feat_l = []
            for feature in features:
                feat_l.append(np.unique(table_current[feature]))
            table_update.set_index(features, inplace=True)
            table_current.set_index(features, inplace=True)

            multi_index_flag = isinstance(table_update.index, pd.MultiIndex)
            if multi_index_flag:
                if not table_update.index.is_monotonic_increasing:
                    table_update.sort_index(inplace=True)
                if not table_current.index.is_monotonic_increasing:
                    table_current.sort_index(inplace=True)
            
            for feature in product(*feat_l):
                print(feature)
                break

            for feature in product(*feat_l):
                
                den = tmp.loc[feature]
                # calculate new weight for this iteration

                if not multi_index_flag:
                    msk = table_update.index == feature[0]
                else:
                    msk = feature

                if den == 0:
                    table_update.loc[msk, weight_col] =\
                        table_current.loc[feature, weight_col] *\
                        xijk.loc[feature]
                else:
                    table_update.loc[msk, weight_col] = \
                        table_current.loc[feature, weight_col].astype(float) * \
                        xijk.loc[feature] / den
                    
            table_update.reset_index(inplace=True)
            table_current.reset_index(inplace=True)
            inc += 1
            feat_l = []

        # Calculate the max convergence rate
        max_conv = 0
        inc = 0
        for features in dimensions:
            tmp = table_update.groupby(features)[weight_col].sum()
            ori_ijk = aggregates[inc]
            temp_conv = max(abs(tmp / ori_ijk - 1))
            if temp_conv > max_conv:
                max_conv = temp_conv
            inc += 1

                

(0.0,)
(0.0,)
(0.0,)
('39010',)


AttributeError: 'numpy.float64' object has no attribute 'loc'

In [26]:
table_current.loc[feature]

flag_air_conditioning                     0.0
flag_garage                               1.0
flag_pertinenza                           1.0
energy_class             Missing_energy_class
COD_CAT                                   A02
anno_costruzione                    2005_2025
log_mq_bin                                  3
log_price_bin                             2.0
count                                 3.62419
Name: 39017, dtype: object

In [76]:
model = ipfn.ipfn(original = seed, aggregates = aggregates,
             dimensions=dimensions, weight_col = "count",
             convergence_rate=1e-5, max_iteration=500)
result = model.iteration()
# weights = result.df

AttributeError: 'numpy.float64' object has no attribute 'loc'

In [95]:
df.value_counts().reset_index().assign(count = lambda x: x["count"] / x["count"].sum() * 1568.0)

,flag_garage,flag_pertinenza,flag_air_conditioning,energy_class,COD_CAT,anno_costruzione,x_bin,y_bin,log_mq_bin,log_price_bin,count
0,0.0,0.0,0.0,Missing_energy_class,A02,1500_1965,1,1,0,0.0,27.787342
1,1.0,0.0,0.0,Missing_energy_class,A02,1985_2005,4,4,4,4.0,19.848101
2,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,4,4,4,4.0,15.878481
3,0.0,0.0,0.0,Missing_energy_class,A02,1985_2005,4,4,0,0.0,11.908861
4,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,1,2,2,2.0,11.908861
...,...,...,...,...,...,...,...,...,...,...,...
327,0.0,1.0,0.0,Missing_energy_class,A02,1965_1985,0,1,0,0.0,3.969620
328,0.0,1.0,0.0,Missing_energy_class,A02,1500_1965,3,1,3,3.0,3.969620
329,0.0,1.0,0.0,Missing_energy_class,A02,1500_1965,3,1,3,2.0,3.969620
330,0.0,1.0,0.0,Missing_energy_class,A02,1500_1965,2,0,3,1.0,3.969620


In [99]:
result.sample(n = 10, weights = result["count"])

,log_price_bin,flag_garage,flag_pertinenza,flag_air_conditioning,energy_class,COD_CAT,anno_costruzione,x_bin,y_bin,log_mq_bin,count
268,0.0,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,4,3,0,3.753142
246,4.0,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,1,2,3,6.099109
16,4.0,0.0,0.0,0.0,Missing_energy_class,A02,1500_1965,2,2,3,3.396339
81,0.0,0.0,0.0,0.0,Missing_energy_class,A02,1985_2005,4,4,0,11.830792
258,2.0,1.0,0.0,0.0,Missing_energy_class,A02,2005_2025,3,2,0,4.199900
58,1.0,0.0,0.0,0.0,Missing_energy_class,A02,1965_1985,3,1,3,4.762042
28,3.0,0.0,0.0,0.0,Missing_energy_class,A02,1965_1985,0,0,4,4.913030
312,2.0,1.0,1.0,0.0,Missing_energy_class,A02,2005_2025,0,3,2,4.196578
208,2.0,1.0,0.0,0.0,Missing_energy_class,A02,1985_2005,2,0,3,3.700037
48,4.0,0.0,0.0,0.0,Missing_energy_class,A02,1965_1985,2,1,4,4.444800


In [ ]:

# 1. Load your df as given

# 2. Create bins & marginals
df['x_bin'] = pd.qcut(df['x'], q = 10, labels=False)
df['y_bin'] = pd.qcut(df['y'], q = 10, labels=False)

agg_x = df['x_bin'].value_counts().sort_index()
agg_y = df['y_bin'].value_counts().sort_index()
agg_bool1 = df['bool_1'].value_counts().sort_index()
# [... repeat for other variables ...]

aggregates = [agg_x.values, agg_y.values, agg_bool1.values, ...]
dimensions = [['x_bin'], ['y_bin'], ['bool_1'], ...]

# 3. Run IPF
orig = df[['x_bin','y_bin','bool_1', ...]]  # only constraint columns
ipf = ipfn(orig, aggregates, dimensions)
res = ipf.iteration(max_iteration=500, convergence_rate=1e-5)
weights = res.df

# 4. Weighted sampling
weights['prob'] = weights['weight'] / weights['weight'].sum()
synthetic_df = df.sample(
    n=len(df),
    replace=True,
    weights=weights['prob']
).reset_index(drop=True)


NameError: name 'df' is not defined